In [2]:
# === BIBLIOTECAS NECESSÁRIAS ===
# Manipulação de dados e cálculos numéricos
import pandas as pd 
import numpy as np  
# Visualização estática (gráficos tradicionais)
import matplotlib.pyplot as plt 
import seaborn as sns            
# Manipulação de datas
from datetime import datetime  
# Visualização interativa e dinâmica
import plotly.express as px     
import plotly.graph_objects as go 
from plotly.subplots import make_subplots 
# Manipulação de arquivos e sistemas
import glob
import os  
# Expressões regulares
import re 

In [10]:
# 1) Carregar e concatenar os dados dos processos judiciais da pasta uploads
# Exibição no console
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 200)

# Listar os arquivos CSV na pasta 'uploads'
arquivos_csv = glob.glob('uploads/processos_*.csv')

# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
tamanho_total_bytes = 0   # Somar os tamanhos dos arquivos
for arquivo in arquivos_csv:
    # Extrair o ano do nome do arquivo
    ano = int(arquivo.split('_')[-1].split('.')[0])
    
    # Soma o tamanho de cada arquivo à variável
    tamanho_total_bytes += os.path.getsize(arquivo)
    
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    df_ano['ano_arquivo'] = ano  # Adicionar coluna com o ano do arquivo
    dfs.append(df_ano)

print("\n=== Arquivo carregado com sucesso! ===")
df_juridico = pd.concat(dfs, ignore_index=True)
print("Dimensões (linhas, colunas):", df_juridico.shape)

# Tamanho do arquivo
file_size_mb = tamanho_total_bytes / (1024**2)
print(f"Tamanho total dos arquivos CSV no disco: {file_size_mb:.2f} MB")

# Guardar uma cópia de trabalho
df = df_juridico.copy()


=== Arquivo carregado com sucesso! ===
Dimensões (linhas, colunas): (4944351, 11)
Tamanho total dos arquivos CSV no disco: 726.09 MB


In [12]:
# 2) Tratamento dos Dados para Advogados
# Tratamento das colunas
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['ano_distribuicao'] = df['data_distribuicao'].dt.year # Criar coluna de ano de distribuição
df['is_segredo_justica'] = df['is_segredo_justica'].astype(bool)

# Tratamento dos números de OAB
def is_oab_valida(oab):
    """
    Verifica se um número de OAB é válido seguindo o formato:
    NÚMEROS + LETRA + ESPAÇO + UF. Ex: '2153421N GO'
    """
    if not isinstance(oab, str) or not oab.strip():
        return False
    oab_limpa = oab.upper().strip()
    ufs_validas = ['GO', 'DF', 'SP', 'RJ', 'MG', 'RS', 'SC', 'PR', 'BA', 'PE',
                  'CE', 'MA', 'ES', 'AL', 'SE', 'PB', 'RN', 'PI', 'MT', 'MS', 
                  'TO', 'PA', 'AP', 'AM', 'RR', 'AC', 'RO'
                  ]
    padrao_regex = re.compile(f"^[1-9]\\d*[A-Z]\\s({'|'.join(ufs_validas)})$")
    return bool(padrao_regex.match(oab_limpa))

# Aplicar a validação de OAB
df['oab_valida'] = df['oab'].apply(is_oab_valida)

# Contar e exibir a quantidade de OABs inválidas
registros_invalidos = df[df['oab_valida'] == False]
qtd_invalidos = len(registros_invalidos)

print("\n=== Validação de Registros de OAB ===")
print(f"Total de registros com OAB em formato inválido ou nulo: {qtd_invalidos}")

if qtd_invalidos > 0:
    exemplos_invalidos = registros_invalidos['oab'].unique()
    print(f"Exemplos de OABs inválidas: {exemplos_invalidos}")
print("\n" + "="*100 + "\n")

# Dataframe com apenas OABs válidas
df_validos = df[df['oab_valida'] == True].copy()
print("\n=== Dataframe com apenas OAB's válidas ===")
display(df_validos.head())



=== Validação de Registros de OAB ===
Total de registros com OAB em formato inválido ou nulo: 20335
Exemplos de OABs inválidas: ['GO' '0A RJ' '0A MT' '0S GO' '0A MG' '0S MT' '0S DF' '0S MA' '0A PE' 'TO'
 '0A MS' '0S AL' '0S CE' '0S SC' '0S MG' '0S MS' '0S RS' '0S RN' '0A TO'
 '0A SE' '0S PE' '0A RS' '0S TO']



=== Dataframe com apenas OAB's válidas ===


,processo,data_distribuicao,data_baixa,entrancia,comarca,serventia,is_segredo_justica,codg_classe,oab,codg_assunto,ano_arquivo,ano_distribuicao,oab_valida
0,0119071.75.2004.8.09.0051,2022-05-25,2022-06-30,FINAL,GOIÂNIA,2ª Vara Cível,False,7.0,19365N GO,10671,2022,2022,True
1,0189657.40.2004.8.09.0051,2022-06-02,2024-01-22,FINAL,GOIÂNIA,31ª Vara Cível,False,7.0,10647N GO,10671,2022,2022,True
2,0189657.40.2004.8.09.0051,2022-06-02,2024-01-22,FINAL,GOIÂNIA,31ª Vara Cível,False,7.0,167974A SP,10671,2022,2022,True
3,0189657.40.2004.8.09.0051,2022-06-02,2024-01-22,FINAL,GOIÂNIA,31ª Vara Cível,False,7.0,20763N GO,10671,2022,2022,True
4,0197944.89.2004.8.09.0051,2022-06-07,2022-10-07,FINAL,GOIÂNIA,22ª Vara Cível,False,7.0,20785N GO,10671,2022,2022,True
